### Models für Mimikanalyse : ResNet50 , Resnet101,  VGG16, VGG19, InceptionV3, DenseNet 169 und EfficientNetB7 ohne Dataaugmenation

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
import tensorflow as tf

print("TensorFlow-Version:", tf.__version__)

# Liste der verfügbaren Geräte abrufen
available_devices = tf.config.list_physical_devices()
print("Verfügbare Geräte:", available_devices)

# Überprüfen, ob eine GPU verfügbar ist
is_gpu_available = tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)
print("Ist eine GPU verfügbar:", is_gpu_available)

# Spezifisch nach GPUs suchen
gpus = tf.config.list_physical_devices('GPU')
print("GPUs:", gpus)

TensorFlow-Version: 2.10.1
Verfügbare Geräte: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Ist eine GPU verfügbar: True
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
# TensorBoard Callback einrichten
import datetime

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

#### ResNet50

In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import preprocess_input

# Betitelung für Tensorboard
experiment_name = "ResNet50"

# Funktion zur Vorbereitung der Bilder
def preprocess_dataset(ds):
    return ds.map(lambda x, y: (preprocess_input(x), y))

# Trainingsdaten laden und vorbereiten
train_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

# Preprocess
train_ds = preprocess_dataset(train_ds)

# Validierungsdaten laden und vorbereiten
val_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32)


val_ds = preprocess_dataset(val_ds)

# Vortrainiertes Modell laden
base_model = ResNet50(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Einfrieren der Basis-Schichten

# Modell anpassen
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x) # Dropout zur Regulierung von Overfitting
outputs = Dense(4, activation='softmax')(x)  # 4 Klassen für die Emotionen
model = Model(inputs, outputs)

# Modell kompilieren und trainieren
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Setze den Pfad für die TensorBoard Logs
log_dir = "logs/fit/" + experiment_name + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, update_freq="epoch")

model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=9, 
    callbacks=[tensorboard_callback]  # TensorBoard Callback hinzufügen
)

# Modell mit Testdaten testen
test_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions_Test",
    image_size=(224, 224),
    batch_size=32)
test_ds = preprocess_dataset(test_ds)

# Modell evaluieren
test_loss, test_accuracy = model.evaluate(test_ds)

# Pfad für Test-Logs setzen
test_log_dir = "logs/fit/" + experiment_name + "_test_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Summary Writer für Testdaten erstellen
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

# Testmetriken in TensorBoard loggen
with test_summary_writer.as_default():
    tf.summary.scalar('test_loss', test_loss, step=0)
    tf.summary.scalar('test_accuracy', test_accuracy, step=0)


Found 2508 files belonging to 4 classes.
Using 2007 files for training.
Found 2508 files belonging to 4 classes.
Using 501 files for validation.
Epoch 1/9
63/63 [==============================] - 23s 195ms/step - loss: 1.0498 - accuracy: 0.6886 - val_loss: 0.3518 - val_accuracy: 0.9022
Epoch 2/9
63/63 [==============================] - 20s 322ms/step - loss: 0.3020 - accuracy: 0.8924 - val_loss: 0.1893 - val_accuracy: 0.9361
Epoch 3/9
63/63 [==============================] - 19s 296ms/step - loss: 0.1817 - accuracy: 0.9402 - val_loss: 0.1889 - val_accuracy: 0.9421
Epoch 4/9
63/63 [==============================] - 18s 278ms/step - loss: 0.1509 - accuracy: 0.9532 - val_loss: 0.1171 - val_accuracy: 0.9661
Epoch 5/9
63/63 [==============================] - 20s 315ms/step - loss: 0.0981 - accuracy: 0.9681 - val_loss: 0.1487 - val_accuracy: 0.9421
Epoch 6/9
63/63 [==============================] - 12s 182ms/step - loss: 0.0923 - accuracy: 0.9696 - val_loss: 0.1574 - val_accuracy: 0.9301
Epo

#### ResNet101

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import preprocess_input

# Betitelung für Tensorboard
experiment_name = "ResNet101"

# Funktion zur Vorbereitung der Bilder
def preprocess_dataset(ds):
    return ds.map(lambda x, y: (preprocess_input(x), y))

# Trainingsdaten laden und vorbereiten
train_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

# Preprocess
train_ds = preprocess_dataset(train_ds)

# Validierungsdaten laden und vorbereiten
val_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

val_ds = preprocess_dataset(val_ds)

# Vortrainiertes Modell laden
base_model = ResNet101(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Einfrieren der Basis-Schichten

# Modell anpassen
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x) # Dropout zur Regulierung von Overfitting
outputs = Dense(4, activation='softmax')(x)  # 4 Klassen für die Emotionen
model = Model(inputs, outputs)

# Modell kompilieren und trainieren
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Setze den Pfad für die TensorBoard Logs
log_dir = "logs/fit/" + experiment_name + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, update_freq="epoch")

model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=9, 
    callbacks=[tensorboard_callback]  # TensorBoard Callback hinzufügen
)

# Modell mit Testdaten testen
test_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions_Test",
    image_size=(224, 224),
    batch_size=32)
test_ds = preprocess_dataset(test_ds)

# Modell evaluieren
test_loss, test_accuracy = model.evaluate(test_ds)

# Pfad für Test-Logs setzen
test_log_dir = "logs/fit/" + experiment_name + "_test_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Summary Writer für Testdaten erstellen
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

# Testmetriken in TensorBoard loggen
with test_summary_writer.as_default():
    tf.summary.scalar('test_loss', test_loss, step=0)
    tf.summary.scalar('test_accuracy', test_accuracy, step=0)

Found 2508 files belonging to 4 classes.
Using 2007 files for training.
Found 2508 files belonging to 4 classes.
Using 501 files for validation.
Epoch 1/9
63/63 [==============================] - 16s 198ms/step - loss: 0.9163 - accuracy: 0.7150 - val_loss: 0.2508 - val_accuracy: 0.9401
Epoch 2/9
63/63 [==============================] - 12s 177ms/step - loss: 0.2863 - accuracy: 0.8909 - val_loss: 0.2823 - val_accuracy: 0.9022
Epoch 3/9
63/63 [==============================] - 12s 183ms/step - loss: 0.1688 - accuracy: 0.9387 - val_loss: 0.1529 - val_accuracy: 0.9461
Epoch 4/9
63/63 [==============================] - 11s 169ms/step - loss: 0.1531 - accuracy: 0.9402 - val_loss: 0.0973 - val_accuracy: 0.9661
Epoch 5/9
63/63 [==============================] - 11s 172ms/step - loss: 0.0881 - accuracy: 0.9701 - val_loss: 0.0970 - val_accuracy: 0.9561
Epoch 6/9
63/63 [==============================] - 12s 180ms/step - loss: 0.0881 - accuracy: 0.9691 - val_loss: 0.1747 - val_accuracy: 0.9102
Epo

#### VGG16

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import preprocess_input

# Betitelung für Tensorboard
experiment_name = "VGG16"

# Funktion zur Vorbereitung der Bilder
def preprocess_dataset(ds):
    return ds.map(lambda x, y: (preprocess_input(x), y))

# Trainingsdaten laden und vorbereiten
train_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

# Preprocess
train_ds = preprocess_dataset(train_ds)

# Validierungsdaten laden und vorbereiten
val_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

val_ds = preprocess_dataset(val_ds)

# Vortrainiertes Modell laden
base_model = VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Einfrieren der Basis-Schichten

# Modell anpassen
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x) # Dropout zur Regulierung von Overfitting
outputs = Dense(4, activation='softmax')(x)  # 4 Klassen für die Emotionen
model = Model(inputs, outputs)

# Modell kompilieren und trainieren
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Setze den Pfad für die TensorBoard Logs
log_dir = "logs/fit/" + experiment_name + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, update_freq="epoch")

model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=9, 
    callbacks=[tensorboard_callback]  # TensorBoard Callback hinzufügen
)

# Modell mit Testdaten testen
test_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions_Test",
    image_size=(224, 224),
    batch_size=32)
test_ds = preprocess_dataset(test_ds)

# Modell evaluieren
test_loss, test_accuracy = model.evaluate(test_ds)

# Pfad für Test-Logs setzen
test_log_dir = "logs/fit/" + experiment_name + "_test_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Summary Writer für Testdaten erstellen
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

# Testmetriken in TensorBoard loggen
with test_summary_writer.as_default():
    tf.summary.scalar('test_loss', test_loss, step=0)
    tf.summary.scalar('test_accuracy', test_accuracy, step=0)

Found 2508 files belonging to 4 classes.
Using 2007 files for training.
Found 2508 files belonging to 4 classes.
Using 501 files for validation.
Epoch 1/9
63/63 [==============================] - 19s 230ms/step - loss: 0.7084 - accuracy: 0.7573 - val_loss: 0.1892 - val_accuracy: 0.9281
Epoch 2/9
63/63 [==============================] - 9s 133ms/step - loss: 0.2095 - accuracy: 0.9208 - val_loss: 0.1299 - val_accuracy: 0.9561
Epoch 3/9
63/63 [==============================] - 9s 134ms/step - loss: 0.1068 - accuracy: 0.9601 - val_loss: 0.1381 - val_accuracy: 0.9561
Epoch 4/9
63/63 [==============================] - 9s 137ms/step - loss: 0.1158 - accuracy: 0.9601 - val_loss: 0.0811 - val_accuracy: 0.9721
Epoch 5/9
63/63 [==============================] - 9s 134ms/step - loss: 0.0676 - accuracy: 0.9776 - val_loss: 0.0756 - val_accuracy: 0.9760
Epoch 6/9
63/63 [==============================] - 9s 134ms/step - loss: 0.0523 - accuracy: 0.9796 - val_loss: 0.0494 - val_accuracy: 0.9860
Epoch 7/

#### VGG19

In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import preprocess_input

# Betitelung für Tensorboard
experiment_name = "VGG19"

# Funktion zur Vorbereitung der Bilder
def preprocess_dataset(ds):
    return ds.map(lambda x, y: (preprocess_input(x), y))

# Trainingsdaten laden und vorbereiten
train_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

# Preprocess
train_ds = preprocess_dataset(train_ds)

# Validierungsdaten laden und vorbereiten
val_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

val_ds = preprocess_dataset(val_ds)

# Vortrainiertes Modell laden
base_model = VGG19(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Einfrieren der Basis-Schichten

# Modell anpassen
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x) # Dropout zur Regulierung von Overfitting
outputs = Dense(4, activation='softmax')(x)  # 4 Klassen für die Emotionen
model = Model(inputs, outputs)

# Modell kompilieren und trainieren
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Setze den Pfad für die TensorBoard Logs
log_dir = "logs/fit/" + experiment_name + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, update_freq="epoch")

model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=9, 
    callbacks=[tensorboard_callback]  # TensorBoard Callback hinzufügen
)

# Modell mit Testdaten testen
test_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions_Test",
    image_size=(224, 224),
    batch_size=32)
test_ds = preprocess_dataset(test_ds)

# Modell evaluieren
test_loss, test_accuracy = model.evaluate(test_ds)

# Pfad für Test-Logs setzen
test_log_dir = "logs/fit/" + experiment_name + "_test_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Summary Writer für Testdaten erstellen
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

# Testmetriken in TensorBoard loggen
with test_summary_writer.as_default():
    tf.summary.scalar('test_loss', test_loss, step=0)
    tf.summary.scalar('test_accuracy', test_accuracy, step=0)

Found 2508 files belonging to 4 classes.
Using 2007 files for training.
Found 2508 files belonging to 4 classes.
Using 501 files for validation.
Epoch 1/9
63/63 [==============================] - 11s 171ms/step - loss: 0.7866 - accuracy: 0.7389 - val_loss: 0.2140 - val_accuracy: 0.9242
Epoch 2/9
63/63 [==============================] - 10s 156ms/step - loss: 0.1786 - accuracy: 0.9362 - val_loss: 0.2229 - val_accuracy: 0.9281
Epoch 3/9
63/63 [==============================] - 10s 157ms/step - loss: 0.1220 - accuracy: 0.9591 - val_loss: 0.1787 - val_accuracy: 0.9401
Epoch 4/9
63/63 [==============================] - 10s 157ms/step - loss: 0.1032 - accuracy: 0.9646 - val_loss: 0.1086 - val_accuracy: 0.9721
Epoch 5/9
63/63 [==============================] - 10s 156ms/step - loss: 0.0662 - accuracy: 0.9766 - val_loss: 0.1008 - val_accuracy: 0.9701
Epoch 6/9
63/63 [==============================] - 10s 157ms/step - loss: 0.0597 - accuracy: 0.9801 - val_loss: 0.0984 - val_accuracy: 0.9741
Epo

#### InceptionNetV3

In [12]:
import datetime
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input

# Betitelung für Tensorboard
experiment_name = "InceptionV3"

# Funktion zur Vorbereitung der Bilder
def preprocess_dataset(ds):
    return ds.map(lambda x, y: (preprocess_input(x), y))

# Trainingsdaten laden und vorbereiten
train_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(299,299),
    batch_size=32)

train_ds = preprocess_dataset(train_ds)


# Validierungsdaten laden und vorbereiten
val_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(299, 299),
    batch_size=32)

val_ds = preprocess_dataset(val_ds)

# Vortrainiertes Modell laden
base_model = InceptionV3(input_shape=(299, 299, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Einfrieren der Basis-Schichten


# Modell anpassen
inputs = tf.keras.Input(shape=(299, 299, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
outputs = Dense(4, activation='softmax')(x)  # 4 Klassen für Emotionen
model = Model(inputs, outputs)

# Modell kompilieren und trainieren
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Setze den Pfad für die TensorBoard Logs
log_dir = "logs/fit/" + experiment_name + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, update_freq="epoch")

model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=9, 
    callbacks=[tensorboard_callback]  # TensorBoard Callback hinzufügen
)

# Modell mit Testdaten testen
test_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions_Test",
    image_size=(299, 299),
    batch_size=32)
test_ds = preprocess_dataset(test_ds)

# Modell evaluieren
test_loss, test_accuracy = model.evaluate(test_ds)

# Pfad für Test-Logs setzen
test_log_dir = "logs/fit/" + experiment_name + "_test_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Summary Writer für Testdaten erstellen
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

# Testmetriken in TensorBoard loggen
with test_summary_writer.as_default():
    tf.summary.scalar('test_loss', test_loss, step=0)
    tf.summary.scalar('test_accuracy', test_accuracy, step=0)

Found 2508 files belonging to 4 classes.
Using 2007 files for training.
Found 2508 files belonging to 4 classes.
Using 501 files for validation.
Epoch 1/9
63/63 [==============================] - 25s 289ms/step - loss: 1.2272 - accuracy: 0.5790 - val_loss: 0.5846 - val_accuracy: 0.8224
Epoch 2/9
63/63 [==============================] - 14s 219ms/step - loss: 0.4772 - accuracy: 0.8371 - val_loss: 0.3114 - val_accuracy: 0.9202
Epoch 3/9
63/63 [==============================] - 15s 224ms/step - loss: 0.2797 - accuracy: 0.9228 - val_loss: 0.2737 - val_accuracy: 0.8922
Epoch 4/9
63/63 [==============================] - 14s 220ms/step - loss: 0.2388 - accuracy: 0.9248 - val_loss: 0.2201 - val_accuracy: 0.9202
Epoch 5/9
63/63 [==============================] - 14s 222ms/step - loss: 0.1524 - accuracy: 0.9532 - val_loss: 0.1649 - val_accuracy: 0.9441
Epoch 6/9
63/63 [==============================] - 14s 221ms/step - loss: 0.1346 - accuracy: 0.9581 - val_loss: 0.1212 - val_accuracy: 0.9621
Epo

#### DenseNet161

In [11]:
import datetime
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.applications import DenseNet169

# Betitelung für Tensorboard
experiment_name = "DenseNet169"

# Funktion zur Vorbereitung der Bilder
def preprocess_dataset(ds):
    return ds.map(lambda x, y: (preprocess_input(x), y))

# Trainingsdaten laden und vorbereiten
train_ds_original = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

train_ds = preprocess_dataset(train_ds_original)


# Validierungsdaten laden und vorbereiten
val_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

val_ds = preprocess_dataset(val_ds)

# Vortrainiertes Modell laden
base_model = DenseNet169(input_shape=(224, 224, 3), include_top=False, weights='imagenet')  # DenseNet 161
base_model.trainable = False  # Einfrieren der Basis-Schichten


# Modell anpassen
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
outputs = Dense(4, activation='softmax')(x)  # 4 Klassen für Ihre Emotionen
model = Model(inputs, outputs)

# Modell kompilieren und trainieren
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Setze den Pfad für die TensorBoard Logs
log_dir = "logs/fit/" + experiment_name + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, update_freq="epoch")

model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=9, 
    callbacks=[tensorboard_callback]  # TensorBoard Callback hinzufügen
)

# Modell mit Testdaten testen
test_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions_Test",
    image_size=(224, 224),
    batch_size=32)
test_ds = preprocess_dataset(test_ds)

# Modell evaluieren
test_loss, test_accuracy = model.evaluate(test_ds)

# Pfad für Test-Logs setzen
test_log_dir = "logs/fit/" + experiment_name + "_test_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Summary Writer für Testdaten erstellen
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

# Testmetriken in TensorBoard loggen
with test_summary_writer.as_default():
    tf.summary.scalar('test_loss', test_loss, step=0)
    tf.summary.scalar('test_accuracy', test_accuracy, step=0)

Found 2508 files belonging to 4 classes.
Using 2007 files for training.
Found 2508 files belonging to 4 classes.
Using 501 files for validation.
Epoch 1/9
63/63 [==============================] - 28s 300ms/step - loss: 0.7953 - accuracy: 0.7120 - val_loss: 0.3377 - val_accuracy: 0.8583
Epoch 2/9
63/63 [==============================] - 16s 248ms/step - loss: 0.2972 - accuracy: 0.8869 - val_loss: 0.2201 - val_accuracy: 0.9441
Epoch 3/9
63/63 [==============================] - 16s 244ms/step - loss: 0.1636 - accuracy: 0.9497 - val_loss: 0.1676 - val_accuracy: 0.9401
Epoch 4/9
63/63 [==============================] - 17s 259ms/step - loss: 0.1359 - accuracy: 0.9547 - val_loss: 0.1435 - val_accuracy: 0.9541
Epoch 5/9
63/63 [==============================] - 17s 272ms/step - loss: 0.0895 - accuracy: 0.9751 - val_loss: 0.1124 - val_accuracy: 0.9661
Epoch 6/9
63/63 [==============================] - 15s 240ms/step - loss: 0.0830 - accuracy: 0.9751 - val_loss: 0.2322 - val_accuracy: 0.9062
Epo

#### EfficientNetB7

In [10]:
import datetime
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import EfficientNetB7  # Update für EfficientNetB7
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.efficientnet import preprocess_input  # Update für EfficientNet-spezifische preprocess_input-Funktion

# Betitelung für Tensorboard
experiment_name = "EfficientNetB7"

# Funktion zur Vorbereitung der Bilder
def preprocess_dataset(ds):
    return ds.map(lambda x, y: (preprocess_input(x), y))

# Trainingsdaten laden und vorbereiten
train_ds_original = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

# Preprocess
train_ds = preprocess_dataset(train_ds_original)


# Validierungsdaten laden und vorbereiten
val_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

val_ds = preprocess_dataset(val_ds)

# Vortrainiertes Modell laden
base_model = EfficientNetB7(input_shape=(224, 224, 3), include_top=False, weights='imagenet')  # Änderung zu EfficientNetB7
base_model.trainable = False  # Einfrieren der Basis-Schichten

# Modell anpassen
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(4, activation='softmax')(x)  # 4 Klassen für Ihre Emotionen
model = Model(inputs, outputs)

# Modell kompilieren und trainieren
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Setze den Pfad für die TensorBoard Logs
log_dir = "logs/fit/" + experiment_name + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, update_freq="epoch")

model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=9, 
    callbacks=[tensorboard_callback]  # TensorBoard Callback hinzufügen
)

# Modell mit Testdaten testen
test_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions_Test",
    image_size=(224, 224),
    batch_size=32)
test_ds = preprocess_dataset(test_ds)

# Modell evaluieren
test_loss, test_accuracy = model.evaluate(test_ds)

# Pfad für Test-Logs setzen
test_log_dir = "logs/fit/" + experiment_name + "_test_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Summary Writer für Testdaten erstellen
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

# Testmetriken in TensorBoard loggen
with test_summary_writer.as_default():
    tf.summary.scalar('test_loss', test_loss, step=0)
    tf.summary.scalar('test_accuracy', test_accuracy, step=0)


Found 2508 files belonging to 4 classes.
Using 2007 files for training.
Found 2508 files belonging to 4 classes.
Using 501 files for validation.
Epoch 1/9
63/63 [==============================] - 39s 475ms/step - loss: 0.6362 - accuracy: 0.7514 - val_loss: 0.3139 - val_accuracy: 0.8583
Epoch 2/9
63/63 [==============================] - 29s 451ms/step - loss: 0.2631 - accuracy: 0.9018 - val_loss: 0.2437 - val_accuracy: 0.9082
Epoch 3/9
63/63 [==============================] - 29s 455ms/step - loss: 0.1900 - accuracy: 0.9258 - val_loss: 0.1951 - val_accuracy: 0.9222
Epoch 4/9
63/63 [==============================] - 27s 429ms/step - loss: 0.1800 - accuracy: 0.9278 - val_loss: 0.1776 - val_accuracy: 0.9341
Epoch 5/9
63/63 [==============================] - 29s 452ms/step - loss: 0.1437 - accuracy: 0.9452 - val_loss: 0.0998 - val_accuracy: 0.9621
Epoch 6/9
63/63 [==============================] - 27s 432ms/step - loss: 0.1131 - accuracy: 0.9576 - val_loss: 0.0879 - val_accuracy: 0.9641
Epo